In [39]:
import os
import polars as pl
import kaggle_evaluation.mcts_inference_server
from pathlib import Path

from sklearn.ensemble import BaggingRegressor, RandomForestRegressor, GradientBoostingRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import OrdinalEncoder

The evaluation API requires that you set up a server which will respond to inference requests. We have already defined the server; you just need write the `predict` function. When we evaluate your submission on the hidden test set the client defined in `mcts_gateway` will run in a different container with direct access to the hidden test set and hand off the data in batches of 100. 

Your code will always have access to the published copies of the files.

In [40]:
# Loading training data
train = pl.read_csv('/kaggle/input/um-game-playing-strength-of-mcts-variants/train.csv')
train[0:2]

Id,GameRulesetName,agent1,agent2,Properties,Format,Time,Discrete,Realtime,Turns,Alternating,Simultaneous,Stochastic,HiddenInformation,Match,Asymmetric,AsymmetricRules,AsymmetricPlayRules,AsymmetricEndRules,AsymmetricForces,AsymmetricSetup,AsymmetricPiecesType,Players,NumPlayers,Simulation,Solitaire,TwoPlayer,Multiplayer,PlayersWithDirections,Cooperation,Team,Coalition,Puzzle,DeductionPuzzle,PlanningPuzzle,Equipment,Container,…,State,StateType,StackState,PieceState,SiteState,SetSiteState,VisitedSites,Variable,SetVar,RememberValues,ForgetValues,SetPending,InternalCounter,SetInternalCounter,PlayerValue,SetHidden,SetInvisible,SetHiddenCount,SetHiddenRotation,SetHiddenState,SetHiddenValue,SetHiddenWhat,SetHiddenWho,Efficiency,CopyContext,Then,ForEachPiece,DoLudeme,Trigger,PlayoutsPerSecond,MovesPerSecond,EnglishRules,LudRules,num_wins_agent1,num_draws_agent1,num_losses_agent1,utility_agent1
i64,str,str,str,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,…,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,f64,f64,str,str,i64,i64,i64,f64
0,"""00Y""","""MCTS-ProgressiveHistory-0.1-MA…","""MCTS-ProgressiveHistory-0.6-Ra…",1,1,1,1,0,1,1,0,0,0,0,0,0,0,0,0,0,0,1,2,0,0,1,0,0,0,0,0,0,0,0,1,1,…,1,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,1,298.07,18877.17,"""Goal: Connect all three edge c…","""(game ""00'Y'"" (players 2) (equ…",4,0,11,-0.466667
1,"""00Y""","""MCTS-ProgressiveHistory-0.1-MA…","""MCTS-UCB1GRAVE-0.6-NST-true""",1,1,1,1,0,1,1,0,0,0,0,0,0,0,0,0,0,0,1,2,0,0,1,0,0,0,0,0,0,0,0,1,1,…,1,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,1,298.07,18877.17,"""Goal: Connect all three edge c…","""(game ""00'Y'"" (players 2) (equ…",5,0,10,-0.333333


In [41]:
train.null_count()

Id,GameRulesetName,agent1,agent2,Properties,Format,Time,Discrete,Realtime,Turns,Alternating,Simultaneous,Stochastic,HiddenInformation,Match,Asymmetric,AsymmetricRules,AsymmetricPlayRules,AsymmetricEndRules,AsymmetricForces,AsymmetricSetup,AsymmetricPiecesType,Players,NumPlayers,Simulation,Solitaire,TwoPlayer,Multiplayer,PlayersWithDirections,Cooperation,Team,Coalition,Puzzle,DeductionPuzzle,PlanningPuzzle,Equipment,Container,…,State,StateType,StackState,PieceState,SiteState,SetSiteState,VisitedSites,Variable,SetVar,RememberValues,ForgetValues,SetPending,InternalCounter,SetInternalCounter,PlayerValue,SetHidden,SetInvisible,SetHiddenCount,SetHiddenRotation,SetHiddenState,SetHiddenValue,SetHiddenWhat,SetHiddenWho,Efficiency,CopyContext,Then,ForEachPiece,DoLudeme,Trigger,PlayoutsPerSecond,MovesPerSecond,EnglishRules,LudRules,num_wins_agent1,num_draws_agent1,num_losses_agent1,utility_agent1
u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,…,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,…,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [42]:
# Target variable
target = 'utility_agent1'

In [43]:
# Checking the number of rows and columns in train dataset
train.shape

(233234, 814)

In [44]:
# creating a dataframe for for target variable as y_train 
y_train = train[target]

In [45]:
# columns to drop
cols_to_drop = ['num_draws_agent1', 'num_losses_agent1', 'num_wins_agent1', target]

In [46]:
# The target variale will be dropped from the train dataset
train = train.drop(cols_to_drop)
train.shape

(233234, 810)

#### Encoding Categorical columns

In [48]:
# Selectin the object columns
obj_cols = train.select(pl.col(pl.String)).columns

# Creating the encoding object
enc = OrdinalEncoder(handle_unknown = 'use_encoded_value', unknown_value= -999, encoded_missing_value = -9999)

enc.fit(train[obj_cols])

OrdinalEncoder(encoded_missing_value=-9999, handle_unknown='use_encoded_value',
               unknown_value=-999)

In [50]:
# Transform the categorical columns in 'train' using the fitted OrdinalEncoder
train_transformed = enc.transform(train[obj_cols])

# Loop through each categorical column and its corresponding encoded values
for e, c in enumerate(obj_cols):
    # Replace the original categorical columns with their encoded values in the 'train' DataFrame
    train = train.with_columns(pl.Series(c, train_transformed[:, e]))


In [52]:
train[obj_cols][:2]

GameRulesetName,agent1,agent2,Behaviour,StateRepetition,Duration,Complexity,BoardCoverage,GameOutcome,StateEvaluation,Clarity,Decisiveness,Drama,MoveEvaluation,StateEvaluationDifference,BoardSitesOccupied,BranchingFactor,DecisionFactor,MoveDistance,PieceNumber,ScoreDifference,EnglishRules,LudRules
f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
0.0,0.0,10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,716.0,0.0
0.0,0.0,45.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,716.0,0.0


In [ ]:
# Define models to test with Bagging
models = {
    'rf': RandomForestRegressor(),
    'gbr': GradientBoostingRegressor(),
}

# Define parameter grids for grid search (using valid parameters)
param_grid = {
    'rf': {'n_estimators': [100, 200], 'min_samples_split': [2, 5]},
    'gbr': {'n_estimators': [100, 200], 'learning_rate': [0.01, 0.1]},
}

In [ ]:
# Perform GridSearchCV for each model
best_models = {}
for model_name in models:
    grid = GridSearchCV(BaggingRegressor(estimator=models[model_name], n_jobs=-1),
                        param_grid[model_name], cv=3, scoring='neg_mean_squared_error')
    grid.fit(train, y_train)
    best_models[model_name] = grid.best_estimator_

    # Output best parameters
    print(f"Best parameters for {model_name}: {grid.best_params_}")

In [ ]:
# Loading training data
test = pl.read_csv('/kaggle/input/um-game-playing-strength-of-mcts-variants/test.csv')
test[0:2]

In [ ]:
def predict(test: pl.DataFrame, sample_sub: pl.DataFrame):
    # Replace this function with your inference code.
    # You can return either a Pandas or Polars dataframe, though Polars is recommended.
    # Each batch of predictions (except the very first) must be returned within 10 minutes.
    return sample_sub.with_columns(pl.col('utility_agent1') + 0.123)

You must run this cell within 15 minutes or the gateway will throw an error. If you need additional time to load your model, you can do so during the very first `predict` call.

In [ ]:
inference_server = kaggle_evaluation.mcts_inference_server.MCTSInferenceServer(predict)

if os.getenv('KAGGLE_IS_COMPETITION_RERUN'):
    inference_server.serve()
else:
    inference_server.run_local_gateway(
        (
            '/kaggle/input/um-game-playing-strength-of-mcts-variants/test.csv',
            '/kaggle/input/um-game-playing-strength-of-mcts-variants/sample_submission.csv'
        )
    )

Note that nothing past `inference_server.serve()` will be run when your submission is evaluated on the hidden test set.